In [1]:
import sys
import os

# Define the path to your folder
folder_path = f'..\\..\\..\\Models\\AutoDiffusion' 

# Add the folder to sys.path
sys.path.append(folder_path)

In [2]:
import numpy as np
#import process_edited as pce
import process_GQ as pce
import autoencoder as ae
import diffusion as diff
import TabDDPMdiff as TabDiff
import pandas as pd
import torch
import os
import time
import numpy as np

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
strings_set = ['diabetes','oil','yeast_ml8_dataset','creditcard_sampled','HTRU','mammography']
Model = 'AutoDiff'
metrics_list = []


for dataset in strings_set:
    print(f"Result Metrics for AutoDiff Autoencoder & ForestDIffusion for {dataset} dataset")
    file_path = f'..\\..\\..\\Datasets\\Original Data\\{dataset}.csv'
    # Read dataframe
    # print(file_path)
    real_df = pd.read_csv(file_path)
    #real_df = real_df.drop('url', axis=1)
    # # Step 2: Inspect the data and check for class imbalance
    # # Assuming the last column is the label, and the rest are features
    X = real_df.iloc[:, :-1].values  # Features
    y = real_df.iloc[:, -1].values  # Labels (binary classification)
    #  # Separate the minority class
    # Find the minority class
    
    real_minortiy = real_df[y == 1]

    threshold = 0.01 # Threshold for mixed-type variables
    parser = pce.DataFrameParser().fit(real_minortiy, threshold)
    ################################################################################################################
    # Auto-encoder hyper-parameters
    device = 'cuda' #@param {'type':'string'}
    n_epochs = 2000 #@param {'type':'integer'}
    eps = 1e-5 #@param {type:"number"}
    weight_decay = 1e-6 #@param {'type':'number'}
    maximum_learning_rate = 1e-2 #@param {'type':'number'}
    lr = 2e-4 #@param {'type':'number'}
    hidden_size = 250
    num_layers = 3
    batch_size = real_minortiy.shape[0] # Full batch

    ds = ae.train_autoencoder(real_minortiy, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold)
    latent_features = ds[1].detach()

    from ForestDiffusion import ForestDiffusionModel


    # Convert to NumPy array
    array = latent_features.detach().cpu().numpy()
    forest_model = ForestDiffusionModel(array, label_y=None, n_t=50, duplicate_K=100, bin_indexes=[], cat_indexes=[], int_indexes=[], diffusion_type='flow', n_jobs=-1)
    minority_fake = forest_model.generate(batch_size=len(real_minortiy)) # Adjust the batch size to create a balanced dataset
    sample=torch.tensor(minority_fake, dtype=torch.float32)
    sample.shape
    gen_output = ds[0](sample, ds[2], ds[3])
    gen_df = pce.convert_to_table(real_minortiy, gen_output, threshold)

    output_directory =  f'..\\..\\..\\Datasets\\Synthetic Data\\'
    filename = f'{Model}+Forest_{dataset}_Synthetic.csv'
    output_file = os.path.join(output_directory, filename)
    gen_df.to_csv(output_file, index=False) 


    # Select a random sample of the generated data
    selected_samples = gen_df.sample(n=min(100,gen_df.shape[0]), random_state=42)  # For reproducibility
    # Syn _df will be the dataset after augmentation
    syn_df = pd.concat([real_df, selected_samples], ignore_index=True)


    augmented_output_directory =  f'..\\..\\..\\Datasets\\Augmented Data\\'
    filename = f'{Model}+Forest_{dataset}_Augmented.csv'
    augmented_output_file = os.path.join(augmented_output_directory, filename)
    syn_df.to_csv(augmented_output_file, index=False) 

    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
    import numpy as np
    import matplotlib.pyplot as plt
    from ForestDiffusion import ForestDiffusionModel
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import recall_score, f1_score

    # real_df = pd.read_csv(filename)
    # syn_filename = f'{string}/{Model}_{string}_Augmented.csv'

    # augmented_df = pd.read_csv(syn_filename)
    augmented_df=syn_df

    X = real_df.iloc[:, :-1].values  # Features
    y = real_df.iloc[:, -1].values 
    # Check and print the original class distribution
    unique, counts = np.unique(y, return_counts=True)
    class_dist_before = dict(zip(unique, counts))
    print(f"Class distribution before augmentation: {class_dist_before}")# Labels (binary classification)

    X_balanced = augmented_df.iloc[:, :-1].values  # Features
    y_balanced = augmented_df.iloc[:, -1].values  # Labels (binary classification)

    # Check and print the Augmented class distribution
    unique, counts = np.unique(y_balanced, return_counts=True)
    class_dist_after = dict(zip(unique, counts))
    print(f"Class distribution after augmentation: {class_dist_after}")

    # Step 6: Split the dataset into training and test sets (original and balanced)
    X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

    # Step 7: Train a simple classifier on both original and generated datasets
    clf_orig = RandomForestClassifier(random_state=42)
    clf_orig.fit(X_train_orig, y_train_orig)

    clf_bal = RandomForestClassifier(random_state=42)
    clf_bal.fit(X_train_bal, y_train_bal)

    # Step 8: Predict and calculate recall and F1 scores
    y_pred_orig = clf_orig.predict(X_test_orig)
    y_pred_bal = clf_bal.predict(X_test_orig)

    prec_orig = precision_score(y_test_orig, y_pred_orig)
    prec_bal = precision_score(y_test_orig, y_pred_bal)
    
    recall_orig = recall_score(y_test_orig, y_pred_orig)
    recalls_bal = recall_score(y_test_orig, y_pred_bal)

    f1_orig = f1_score(y_test_orig, y_pred_orig)
    f1_bal = f1_score(y_test_orig, y_pred_bal)

    # Step 9: Print and store the performance metrics
    # Store metrics in a dictionary
    metrics = {
    "Dataset": dataset,
    "Precision_Original": prec_orig,
    "Precision_Generated": prec_bal,
    "Recall_Original": recall_orig,
    "Recall_Generated": recalls_bal,
    "F1_Original": f1_orig,   
    "F1_Generated": f1_bal,
    "Num_Fake_Samples": len(augmented_df) - len(real_df),
    "Synthetic/Original_Ratio": 100*(len(augmented_df) - len(real_df))/len(real_minortiy)
    }

    # Append the dictionary to the list
    metrics_list.append(metrics)

    print(f"Precision score (original data): {prec_orig:.4f}")
    print(f"Precision score (generated data): {prec_bal:.4f}")
    print(f"Recall score (original data): {recall_orig:.4f}")
    print(f"Recall score (generated data): {recalls_bal:.4f}")
    print(f"F1 score (original data): {f1_orig:.4f}")
    print(f"F1 score (generated data): {f1_bal:.4f}")
    print("Classification Report (original data):\n", classification_report(y_test_orig, y_pred_orig))
    print("Classification Report (generated data):\n", classification_report(y_test_orig, y_pred_bal))


    print(f"Number of fake samples generated: {len(augmented_df)-len(real_df)}")
# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Save the DataFrame to a CSV file
metrics_df.to_csv("Auto_Diff_Forest_different_datasets_metric.csv", index=False)

# Print the DataFrame
print(metrics_df)


Result Metrics for AutoDiff Autoencoder & ForestDIffusion for yeast_ml8_dataset dataset


  0%|          | 0/2000 [00:00<?, ?it/s]

Class distribution before augmentation: {-1: 2239, 1: 178}
Class distribution after augmentation: {-1.0: 2239, 1.0: 278}
Precision score (original data): 0.0000
Precision score (generated data): 1.0000
Recall score (original data): 0.0000
Recall score (generated data): 0.5510
F1 score (original data): 0.0000
F1 score (generated data): 0.7105
Classification Report (original data):
               precision    recall  f1-score   support

          -1       0.93      1.00      0.97       677
           1       0.00      0.00      0.00        49

    accuracy                           0.93       726
   macro avg       0.47      0.50      0.48       726
weighted avg       0.87      0.93      0.90       726

Classification Report (generated data):
               precision    recall  f1-score   support

          -1       0.97      1.00      0.98       677
           1       1.00      0.55      0.71        49

    accuracy                           0.97       726
   macro avg       0.98      0

c:\Users\tawfi\Thesis\env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tawfi\Thesis\env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tawfi\Thesis\env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tawfi\Thesis\env\Lib\site-packages\sklearn\metrics\_classificati

  0%|          | 0/2000 [00:00<?, ?it/s]

...

Class distribution before augmentation: {0: 16259, 1: 1639}
Class distribution after augmentation: {0.0: 16259, 1.0: 1739}
Precision score (original data): 0.9376
Precision score (generated data): 0.9794
Recall score (original data): 0.8354
Recall score (generated data): 0.8786
F1 score (original data): 0.8836
F1 score (generated data): 0.9262
Classification Report (original data):
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      4884
           1       0.94      0.84      0.88       486

    accuracy                           0.98      5370
   macro avg       0.96      0.91      0.94      5370
weighted avg       0.98      0.98      0.98      5370

Classification Report (generated data):
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      4884
           1       0.98      0.88      0.93       486

    accuracy                           0.99      5370
   macro avg       0.98     

  0%|          | 0/2000 [00:00<?, ?it/s]

Class distribution before augmentation: {0: 500, 1: 268}
Class distribution after augmentation: {0.0: 500, 1.0: 368}
Precision score (original data): 0.6386
Precision score (generated data): 0.7204
Recall score (original data): 0.6625
Recall score (generated data): 0.8375
F1 score (original data): 0.6503
F1 score (generated data): 0.7746
Classification Report (original data):
               precision    recall  f1-score   support

           0       0.82      0.80      0.81       151
           1       0.64      0.66      0.65        80

    accuracy                           0.75       231
   macro avg       0.73      0.73      0.73       231
weighted avg       0.76      0.75      0.75       231

Classification Report (generated data):
               precision    recall  f1-score   support

           0       0.91      0.83      0.87       151
           1       0.72      0.84      0.77        80

    accuracy                           0.83       231
   macro avg       0.81      0.83 

  0%|          | 0/2000 [00:00<?, ?it/s]

Class distribution before augmentation: {0: 4000, 1: 50}
Class distribution after augmentation: {0.0: 4000, 1.0: 100}
Precision score (original data): 0.8667
Precision score (generated data): 0.8750
Recall score (original data): 0.7647
Recall score (generated data): 0.8235
F1 score (original data): 0.8125
F1 score (generated data): 0.8485
Classification Report (original data):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1198
           1       0.87      0.76      0.81        17

    accuracy                           1.00      1215
   macro avg       0.93      0.88      0.90      1215
weighted avg       0.99      1.00      0.99      1215

Classification Report (generated data):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1198
           1       0.88      0.82      0.85        17

    accuracy                           1.00      1215
   macro avg       0.94      0.91

In [3]:
import pandas as pd
metrics_df=pd.read_csv("Auto_Diff_Forest_different_datasets_metric.csv")
metrics_df.head(6)

,Dataset,Precision_Original,Recall_Original,F1_Original,Precision_Generated,Recall_Generated,F1_Generated,Num_Fake_Samples
0,yeast_ml8_dataset,0.000000,0.000000,0.000000,1.000000,0.551020,0.710526,100
1,oil,0.600000,0.272727,0.375000,0.875000,0.636364,0.736842,41
2,mammography,0.916667,0.594595,0.721311,0.966102,0.770270,0.857143,100
3,HTRU,0.937644,0.835391,0.883569,0.979358,0.878601,0.926247,100
4,diabetes,0.638554,0.662500,0.650307,0.720430,0.837500,0.774566,100
5,creditcard_sampled,0.866667,0.764706,0.812500,0.875000,0.823529,0.848485,50


In [6]:
metrics_df[metrics_df['Dataset']=='creditcard_sampled'] 

,Dataset,Precision_Original,Recall_Original,F1_Original,Precision_Generated,Recall_Generated,F1_Generated,Num_Fake_Samples
5,creditcard_sampled,0.866667,0.764706,0.8125,0.875,0.823529,0.848485,50


In [4]:
import os
import time
import numpy as np
import sys
# Define the path to your folder
folder_path = f'..\\..\\..\\Models\\AutoDiffusion' 

# Add the folder to sys.path
sys.path.append(folder_path)
strings_set = ['reduced_diabetes']
Model = 'AutoDiff'
metrics_list = []
import numpy as np
#import process_edited as pce
import process_GQ as pce
import autoencoder as ae
import diffusion as diff
import TabDDPMdiff as TabDiff
import pandas as pd
import torch






for dataset in strings_set:
    print(f"Result Metrics for AutoDiff Autoencoder & ForestDIffusion for {dataset} dataset")
    file_path = f'..\\..\\..\\Datasets\\Original Data\\{dataset}.csv'
    # Read dataframe
    # print(file_path)
    real_df = pd.read_csv(file_path)
    #real_df = real_df.drop('url', axis=1)
    # # Step 2: Inspect the data and check for class imbalance
    # # Assuming the last column is the label, and the rest are features
    X = real_df.iloc[:, :-1].values  # Features
    y = real_df.iloc[:, -1].values  # Labels (binary classification)
    #  # Separate the minority class
    # Find the minority class
    
    real_minortiy = real_df[y == 1]

    threshold = 0.01 # Threshold for mixed-type variables
    parser = pce.DataFrameParser().fit(real_minortiy, threshold)
    ################################################################################################################
    # Auto-encoder hyper-parameters
    device = 'cuda' #@param {'type':'string'}
    n_epochs = 2000 #@param {'type':'integer'}
    eps = 1e-5 #@param {type:"number"}
    weight_decay = 1e-6 #@param {'type':'number'}
    maximum_learning_rate = 1e-2 #@param {'type':'number'}
    lr = 2e-4 #@param {'type':'number'}
    hidden_size = 250
    num_layers = 3
    batch_size = real_minortiy.shape[0] # Full batch

    ds = ae.train_autoencoder(real_minortiy, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold)
    latent_features = ds[1].detach()

    from ForestDiffusion import ForestDiffusionModel


    # Convert to NumPy array
    array = latent_features.detach().cpu().numpy()
    forest_model = ForestDiffusionModel(array, label_y=None, n_t=50, duplicate_K=100, bin_indexes=[], cat_indexes=[], int_indexes=[], diffusion_type='flow', n_jobs=-1)
    minority_fake = forest_model.generate(batch_size=60) # Adjust the batch size to create a balanced dataset
    sample=torch.tensor(minority_fake, dtype=torch.float32)
    sample.shape
    gen_output = ds[0](sample, ds[2], ds[3])
    gen_df = pce.convert_to_table(real_minortiy, gen_output, threshold)

    output_directory =  f'..\\..\\..\\Datasets\\Synthetic Data\\'
    filename = f'{Model}+Forest_{dataset}_Synthetic.csv'
    output_file = os.path.join(output_directory, filename)
    gen_df.to_csv(output_file, index=False) 


    # Select a random sample of the generated data
    selected_samples = gen_df.sample(n=min(100,gen_df.shape[0]), random_state=42)  # For reproducibility
    # Syn _df will be the dataset after augmentation
    syn_df = pd.concat([real_df, selected_samples], ignore_index=True)


    # augmented_output_directory =  f'..\\..\\..\\Datasets\\Augmented Data\\'
    # filename = f'{Model}+Forest_{dataset}_Augmented.csv'
    # augmented_output_file = os.path.join(augmented_output_directory, filename)
    # syn_df.to_csv(augmented_output_file, index=False) 

    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
    import numpy as np
    import matplotlib.pyplot as plt
    from ForestDiffusion import ForestDiffusionModel
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import recall_score, f1_score

    # real_df = pd.read_csv(filename)
    # syn_filename = f'{string}/{Model}_{string}_Augmented.csv'

    # augmented_df = pd.read_csv(syn_filename)
    augmented_df=syn_df

    X = real_df.iloc[:, :-1].values  # Features
    y = real_df.iloc[:, -1].values 
    # Check and print the original class distribution
    unique, counts = np.unique(y, return_counts=True)
    class_dist_before = dict(zip(unique, counts))
    print(f"Class distribution before augmentation: {class_dist_before}")# Labels (binary classification)

    X_balanced = augmented_df.iloc[:, :-1].values  # Features
    y_balanced = augmented_df.iloc[:, -1].values  # Labels (binary classification)

    # Check and print the Augmented class distribution
    unique, counts = np.unique(y_balanced, return_counts=True)
    class_dist_after = dict(zip(unique, counts))
    print(f"Class distribution after augmentation: {class_dist_after}")

    # Step 6: Split the dataset into training and test sets (original and balanced)
    X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

    # Step 7: Train a simple classifier on both original and generated datasets
    clf_orig = RandomForestClassifier(random_state=42)
    clf_orig.fit(X_train_orig, y_train_orig)

    clf_bal = RandomForestClassifier(random_state=42)
    clf_bal.fit(X_train_bal, y_train_bal)

    # Step 8: Predict and calculate recall and F1 scores
    y_pred_orig = clf_orig.predict(X_test_orig)
    y_pred_bal = clf_bal.predict(X_test_orig)

    prec_orig = precision_score(y_test_orig, y_pred_orig)
    prec_bal = precision_score(y_test_orig, y_pred_bal)
    
    recall_orig = recall_score(y_test_orig, y_pred_orig)
    recalls_bal = recall_score(y_test_orig, y_pred_bal)

    f1_orig = f1_score(y_test_orig, y_pred_orig)
    f1_bal = f1_score(y_test_orig, y_pred_bal)
    



augmented_df.to_csv('augmented_dataset__(from autodiffusion paper)autoencoder-forest_reduced_diabetes.csv', index=False)

Result Metrics for AutoDiff Autoencoder & ForestDIffusion for reduced_diabetes dataset


  0%|          | 0/2000 [00:00<?, ?it/s]

Class distribution before augmentation: {0: 500, 1: 60}
Class distribution after augmentation: {0.0: 500, 1.0: 120}


In [5]:
print(f"# Result Metrics for Audiff Paper Autoencoder+ForestDiffusion for {dataset} dataset")
print("Classification Report (original data):\n", classification_report(y_test_orig, y_pred_orig))
print("Classification Report (generated data):\n", classification_report(y_test_orig, y_pred_bal))

# Result Metrics for Audiff Paper Autoencoder+ForestDiffusion for reduced_diabetes dataset
Classification Report (original data):
               precision    recall  f1-score   support

           0       0.91      0.97      0.94       151
           1       0.43      0.18      0.25        17

    accuracy                           0.89       168
   macro avg       0.67      0.57      0.60       168
weighted avg       0.86      0.89      0.87       168

Classification Report (generated data):
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       151
           1       0.88      0.88      0.88        17

    accuracy                           0.98       168
   macro avg       0.93      0.93      0.93       168
weighted avg       0.98      0.98      0.98       168

